https://www.kaggle.com/c/state-farm-distracted-driver-detection/overview

In [ ]:
# Hello this is the RAM optimised branch

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import trange

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Preprocessing

In [3]:
X = []
y = []
for i in range(10):
    dir_path =f'data/imgs/train/c{i}'
    files = os.listdir(dir_path)
    for img_name in files[:100]:
        full_path = dir_path + '/' + img_name
        # print(full_path)
        img = np.array(Image.open(full_path))
        # print(img.shape)
        X.append(img)
        y.append(i)

X = torch.tensor(np.array(X)).float().view(-1, 3, 640, 480)
y = torch.tensor(np.array(y)).float()
X.shape, y.shape

(torch.Size([1000, 3, 640, 480]), torch.Size([1000]))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
class DriverDataset(Dataset):
    def __init__(self, x_, y_):
        self.x_ = x_.to(device)
        self.y_ = y_.to(device)
    
    def __len__(self):
        return self.x_.shape[0]
    
    def __getitem__(self, index):
        return self.x_[index], self.y_[index]


In [6]:
train_dataset = DriverDataset(X_train, y_train)
test_dataset = DriverDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128, shuffle=False)

## Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16*158*118,640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        # x -> (n, 3, 640, 480)
        out = self.pool(F.relu(self.conv1(x))) # -> (n, 6, 318, 238)
        out = self.pool(F.relu(self.conv2(out))) # -> (n, 16, 158, 118)
        out = out.view(-1, 16*158*118)
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)

In [8]:
loss_category = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [9]:
model = torch.compile(model)

In [10]:
torch.set_float32_matmul_precision('high')

In [11]:
for epoch in trange(100):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(torch.float32).to(device)
        y_ = F.one_hot(y_.clone().detach().long(), 10).float().to(device)
        y_pred = model(x_)
        loss = loss_category(y_pred, y_)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()

    if (epoch+1)%5 == 0: print(f'Epoch {epoch+1}: {loss.item():3f}')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 5: 113.422081
Epoch 10: 42.444019
Epoch 15: 16.396057
Epoch 20: 17.135454
Epoch 25: 10.446857
Epoch 30: 1.965217
Epoch 35: 0.809937
Epoch 40: 0.256292
Epoch 45: 0.068550
Epoch 50: 0.031468
Epoch 55: 0.019000
Epoch 60: 0.018021
Epoch 65: 0.015868
Epoch 70: 0.018530
Epoch 75: 0.016834
Epoch 80: 0.010620
Epoch 85: 0.010331
Epoch 90: 0.010075
Epoch 95: 0.009085
Epoch 100: 0.010741


In [12]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item(): score += 1
            tot += 1
print('Train acc:', score*100/ tot, '%')

Train acc: 100.0 %


In [13]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(test_loader):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item(): score += 1
            tot += 1
print('Test acc:', score*100/ tot, '%')


Test acc: 73.6 %


Best acc: 75.6 % 

## Inference

In [30]:
X = []
img_names = []
dir_path =f'data/imgs/test'
files = os.listdir(dir_path)
for img_name in files:
        full_path = dir_path + '/' + img_name
        img = np.array(Image.open(full_path))
        X.append(img)
        img_names.append(img_name)

X = torch.tensor(np.array(X)).float().view(-1, 3, 640, 480)
# y = torch.tensor(np.array(y)).float()
X.shape

: 

In [15]:
inference_dataset = DriverDataset(X, torch.ones(X.shape[0]))
inference_loader = DataLoader(inference_dataset,batch_size=25, shuffle=False)

In [26]:
model.eval()
out = []
with torch.no_grad():
    for i, (x_, y_) in enumerate(inference_loader):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        print(y_pred)
        softmax_output = F.softmax(y_pred, dim=1)
        print(softmax_output)

        out.extend(list(softmax_output))

tensor([[-24.5513, -22.0906, -27.2188, -23.7934, -21.0247, -25.9517, -26.1550,
         -22.4751, -22.8813, -19.3225],
        [-19.7832, -23.5516, -26.9503, -22.1621, -28.1541, -26.5279, -29.8482,
         -31.9852, -26.5445, -21.5750],
        [-24.6558, -21.8363, -21.5164, -20.1665, -22.3255, -25.0595, -24.4865,
         -24.4740, -28.1350, -23.2971],
        [-21.5248, -21.9578, -22.4034, -25.4081, -21.0155, -27.3609, -19.9614,
         -28.8097, -25.9258, -23.3549],
        [-21.8685, -19.3560, -23.8500, -26.3459, -25.9063, -24.7508, -22.8844,
         -22.4161, -24.6943, -23.8559],
        [-25.6899, -21.2612, -17.9141, -23.6820, -20.9290, -20.2419, -13.1392,
         -20.0189, -20.1343, -24.5861],
        [-23.7260, -19.6544, -22.1768, -23.0467, -19.4391, -26.0654, -22.4486,
         -21.4960, -21.7900, -18.6052],
        [-30.1710, -25.5797, -24.9702, -28.4200, -28.9681, -29.2012, -25.3949,
         -27.9799, -24.0676, -31.7371],
        [-21.1701, -30.4281, -28.4236, -27.5030,

In [27]:
len(out), len(img_names)

(120, 120)

In [28]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []

for i in out:
    c0.append(i[0].item())
    c1.append(i[1].item())
    c2.append(i[2].item())
    c3.append(i[3].item())
    c4.append(i[4].item())
    c5.append(i[5].item())
    c6.append(i[6].item())
    c7.append(i[7].item())
    c8.append(i[8].item())
    c9.append(i[9].item())

In [29]:
out = {'img':[img for img in img_names], 
       'c0': c0,
       'c1': c1,
       'c2': c2,
       'c3': c3,
       'c4': c4,
       'c5': c5,
       'c6': c6,
       'c7': c7,
       'c8': c8,
       'c9': c9,
       }
df = pd.DataFrame(out)
df.to_csv('data/output.csv', index=False)